# How to use pyELQ
This example is intended to provide a basic overview on how to work with the pyELQ repo. We will set up a basic example where we generate some concentration data and try to estimate the source location and emisson rate of these synthetic sources.

First we import all the required packages.

In [ ]:
import datetime
from copy import deepcopy

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pyelq.component.background import SpatioTemporalBackground
from pyelq.component.error_model import BySensor
from pyelq.component.offset import PerSensor
from pyelq.component.source_model import Normal
from pyelq.coordinate_system import ENU, LLA
from pyelq.dispersion_model.gaussian_plume import GaussianPlume
from pyelq.gas_species import CH4
from pyelq.model import ELQModel
from pyelq.meteorology import Meteorology
from pyelq.plotting.plot import Plot
from pyelq.preprocessing import Preprocessor
from pyelq.sensor.beam import Beam
from pyelq.sensor.sensor import Sensor, SensorGroup
from pyelq.source_map import SourceMap

Next we set up a time axis as well as a reference point which we use in our local coordinate systems.

In [ ]:
time_axis = pd.array(
    pd.date_range(start="2024-01-01 08:00:00", end="2024-01-01 12:00:00", freq="120s"), dtype="datetime64[ns]"
)
nof_observations = time_axis.size
reference_latitude = 0
reference_longitude = 0
reference_altitude = 0

We define a couple of regularly spaced beam sensors by creating a local ENU frame, defining the beam end points in that coordinate frame and next transform the ENU coordinates to LLA coordinates.

In [ ]:
radius = 30
angles = np.linspace(0, 90, 5)
sensor_x = radius * np.cos(angles * np.pi / 180)
sensor_y = radius * np.sin(angles * np.pi / 180)
sensor_z = np.ones_like(sensor_x) * 5.0

In [ ]:
ENU_object = ENU(ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude)
ENU_object.from_array(np.vstack([sensor_x, sensor_y, sensor_z]).T)
LLA_object = ENU_object.to_lla()
LLA_array = LLA_object.to_array()
print(LLA_array)

We create a SensorGroup which contains all the 5 beams we have set up. We set the sensor position (beam start points) to be at the reference latitude and longitude. At an altitude of 3 meters, similar to the beam end points. The beam layout can be seen in the plot. We initialize the concentration and the time attributes of the sensor so we can use it later to calculate the simulated concentration observations.

In [ ]:
nof_sensors = LLA_array.shape[0]
sensor_group = SensorGroup()
for sensor in range(nof_sensors):
    new_sensor = Beam()
    new_sensor.label = f"Beam sensor {sensor}"
    new_sensor.location = LLA(
        latitude=np.array([reference_latitude, LLA_object.latitude[sensor]]),
        longitude=np.array([reference_longitude, LLA_object.longitude[sensor]]),
        altitude=np.array([5.0, LLA_object.altitude[sensor]]),
    )

    new_sensor.time = time_axis
    new_sensor.concentration = np.zeros(nof_observations)
    sensor_group.add_sensor(new_sensor)

Let's also add some point sensors to our SensorGroup

In [ ]:
sensor_x = np.array([5, 20])
sensor_y = np.array([22, 5])
sensor_z = np.ones_like(sensor_x) * 1.0
ENU_object = ENU(ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude)
ENU_object.from_array(np.vstack([sensor_x, sensor_y, sensor_z]).T)
LLA_object = ENU_object.to_lla()
LLA_array = LLA_object.to_array()

nof_sensors = LLA_array.shape[0]
for sensor in range(nof_sensors):
    new_sensor = Sensor()
    new_sensor.label = f"Point sensor {sensor}"
    new_sensor.location = LLA(
        latitude=np.array([LLA_object.latitude[sensor]]),
        longitude=np.array([LLA_object.longitude[sensor]]),
        altitude=np.array([LLA_object.altitude[sensor]]),
    )

    new_sensor.time = time_axis
    new_sensor.concentration = np.zeros(nof_observations)
    sensor_group.add_sensor(new_sensor)

In [ ]:
fig = go.Figure()
fig = sensor_group.plot_sensor_location(fig=fig)
fig.update_layout(
    map_style="open-street-map",
    map_center=dict(lat=reference_latitude, lon=reference_longitude),
    map_zoom=18,
    height=800,
    margin={"r": 0, "l": 0, "b": 0},
)
fig.show()

We use the meteorology object to store the simulated meteorology observations like wind speed and direction and show these in a wind rose plot.

In [ ]:
met_object = Meteorology()
random_generator = np.random.default_rng(0)

met_object.time = time_axis
met_object.wind_direction = np.linspace(0.0, 90.0, nof_observations) + random_generator.normal(
    loc=0.0, scale=0.1, size=nof_observations
)
met_object.wind_speed = 4.0 * np.ones_like(met_object.wind_direction) + random_generator.normal(
    loc=0.0, scale=0.1, size=nof_observations
)

met_object.calculate_uv_from_wind_speed_direction()

met_object.temperature = (273.1 + 15.0) * np.ones_like(met_object.wind_direction)
met_object.pressure = 101.325 * np.ones_like(met_object.wind_direction)

met_object.wind_turbulence_horizontal = 5.0 * np.ones_like(met_object.wind_direction)
met_object.wind_turbulence_vertical = 5.0 * np.ones_like(met_object.wind_direction)

fig = met_object.plot_polar_hist()
fig.update_layout(height=400, margin={"r": 0, "l": 0})
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=time_axis, y=met_object.wind_direction, mode="markers", name="Wind direction"))
fig.update_layout(height=400, margin={"r": 0, "l": 0}, title="Wind Direction [deg]")
fig.show()

We set up a source map which contains the location information of the simulated sources. We define them in a certain location but could also let this object generate sources using for example a latin hypercube design within the specifies site limits.

In [ ]:
source_map = SourceMap()
site_limits = np.array([[0, 30], [0, 30], [0, 3]])
location_object = ENU(
    ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude
)

source_map.generate_sources(
    coordinate_object=location_object, sourcemap_limits=site_limits, sourcemap_type="hypercube", nof_sources=2
)

source_map.location.up = np.array([2.0, 3.0])
source_map.location.east = np.array([10.0, 20.0])
source_map.location.north = np.array([20.0, 15.0])

In [ ]:
fig = go.Figure()
fig = sensor_group.plot_sensor_location(fig=fig)
fig.update_layout(
    map_style="open-street-map",
    map_center=dict(lat=reference_latitude, lon=reference_longitude),
    map_zoom=18,
    height=800,
    margin={"r": 0, "l": 0, "b": 0},
)
fig.add_trace(
    go.Scattermap(
        mode="markers",
        lon=source_map.location.to_lla().longitude,
        lat=source_map.location.to_lla().latitude,
        name="True locations",
        marker=go.scattermap.Marker(color="green", size=10),
    )
)
fig.show()

After defining the gas species we are interested in we set the true emission rates and generate the real observations. We calculate the coupling from each source to each sensor using a Gaussian plume model and multiply this coupling with the emission rates of the respective sources. We add this source contribution to the background (in this case 2.0 ppm) and also add some random (measurement) noise. These simulated observations are populating the concentration attribute of each sensor in the sensorgroup. The resulting data is shown in the plot.

In [ ]:
gas_object = CH4()
dispersion_model = GaussianPlume(source_map=deepcopy(source_map))
true_emission_rates = np.array([[15], [10]])
for current_sensor in sensor_group.values():
    coupling_matrix = dispersion_model.compute_coupling(
        sensor_object=current_sensor,
        meteorology_object=met_object,
        gas_object=gas_object,
        output_stacked=False,
        run_interpolation=False,
    )
    source_contribution = coupling_matrix @ true_emission_rates
    observation = (
        source_contribution.flatten()
        + 2.0
        + random_generator.normal(loc=0.0, scale=0.01, size=current_sensor.nof_observations)
    )
    current_sensor.concentration = observation

In [ ]:
fig = go.Figure()
fig = sensor_group.plot_timeseries(fig=fig)
fig.update_layout(height=800, margin={"r": 0, "t": 10, "l": 0, "b": 0})
fig.show()

In [ ]:
fig = go.Figure()
fig = met_object.plot_polar_scatter(fig=fig, sensor_object=sensor_group)
fig.update_layout(height=400, margin={"r": 0, "l": 0})
fig.show()

Normally these raw datasets need some preprocessing like smoothing the data and making sure all the time axes align. Therefore we created the preprocessor class. We show the functionality to apply an aggregate function on the data within the user specified time bins and also show how to apply a wind filter, even though the wind speeds we simulated are most likely all larger than the lower limit.

In [ ]:
analysis_time_range = [datetime.datetime(2024, 1, 1, 8, 0, 0), datetime.datetime(2024, 1, 1, 12, 0, 0)]

smoothing_period = 10 * 60

time_bin_edges = pd.array(
    pd.date_range(analysis_time_range[0], analysis_time_range[1], freq=f"{smoothing_period}s"), dtype="datetime64[ns]"
)

prepocessor_object = Preprocessor(
    time_bin_edges=time_bin_edges, sensor_object=sensor_group, met_object=met_object, aggregate_function="median"
)

min_wind_speed = 0.05
prepocessor_object.filter_on_met(filter_variable=["wind_speed"], lower_limit=[min_wind_speed], upper_limit=[np.inf])

In [ ]:
fig = go.Figure()
fig = prepocessor_object.sensor_object.plot_timeseries(fig=fig)
fig.update_layout(height=800, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

We set up the different parameters for our MCMC model

In [ ]:
source_model = Normal()
source_model.emission_rate_mean = np.array([0], ndmin=1)
source_model.initial_precision = np.array([1 / (2.5**2)], ndmin=1)
source_model.reversible_jump = True
source_model.rate_num_sources = 1.0
source_model.dispersion_model = dispersion_model
source_model.update_precision = False
source_model.site_limits = site_limits
source_model.coverage_detection = 0.1  # ppm
source_model.coverage_test_source = 3.0  # kg/hr

background = SpatioTemporalBackground()
background.n_time = None
background.mean_bg = 2.0
background.spatial_dependence = True
background.initial_precision = 1 / np.power(3e-4, 2)
background.precision_time_0 = 1 / np.power(0.1, 2)
background.spatial_correlation_param = 25.0
background.update_precision = False

offset_model = PerSensor()
offset_model.update_precision = False
offset_model.initial_precision = 1 / (0.001) ** 2

error_model = BySensor()
error_model.initial_precision = 1 / (0.1) ** 2
error_model.prior_precision_shape = 1e-2
error_model.prior_precision_rate = 1e-2

We create an instance of the ELQModel class which forms the interface with the MCMC repo and run the MCMC algorithm. Finally we plot the results

In [ ]:
elq_model = ELQModel(
    sensor_object=prepocessor_object.sensor_object,
    meteorology=prepocessor_object.met_object,
    gas_species=gas_object,
    background=background,
    source_model=source_model,
    error_model=error_model,
    offset_model=offset_model,
)
elq_model.n_iter = 5000

elq_model.initialise()

elq_model.to_mcmc()
elq_model.run_mcmc()
elq_model.from_mcmc()

Finally plotting the results in a separate cell. Note that you can plot all plots in 1 go as well using `plotter.show_all()`

In [ ]:
burn_in = elq_model.n_iter - 1000

plotter = Plot()

plotter.plot_quantification_results_on_map(
    model_object=elq_model, bin_size_x=1, bin_size_y=1, normalized_count_limit=0.1, burn_in=burn_in
)

plotter.plot_fitted_values_per_sensor(
    mcmc_object=elq_model.mcmc, sensor_object=elq_model.sensor_object, burn_in=burn_in
)

true_source_location_trace = go.Scattermap(
    mode="markers",
    lon=source_map.location.to_lla().longitude,
    lat=source_map.location.to_lla().latitude,
    name="True locations",
    marker=go.scattermap.Marker(color="green", size=10),
)

In [ ]:
plotter.figure_dict["fitted_values"].update_layout(height=800, margin={"r": 0, "t": 50, "l": 0, "b": 0}).show()

Note we could have also used this call to plot the fitted values figure:
`plotter = elq_model.plot_fitted_values(plot=plotter)`

In [ ]:
plotter = elq_model.plot_fitted_values(plot=plotter)
plotter.figure_dict["fitted_values"].update_layout(height=800, margin={"r": 0, "t": 50, "l": 0, "b": 0}).show()

In [ ]:
plotter.figure_dict["count_map"].add_trace(true_source_location_trace).update_traces(showlegend=True)
plotter.figure_dict["count_map"].update_layout(height=800, margin={"r": 0, "t": 50, "l": 0, "b": 0}, map_zoom=19)
plotter.figure_dict["count_map"].show()

In [ ]:
plotter.figure_dict["iqr_map"].add_trace(true_source_location_trace).update_traces(showlegend=True)
plotter.figure_dict["iqr_map"].update_layout(height=800, margin={"r": 0, "t": 50, "l": 0, "b": 0}, map_zoom=19)
plotter.figure_dict["iqr_map"].show()

In [ ]:
plotter.figure_dict["median_map"].add_trace(true_source_location_trace).update_traces(showlegend=True)
plotter.figure_dict["median_map"].update_layout(height=800, margin={"r": 0, "t": 50, "l": 0, "b": 0}, map_zoom=19)
plotter.figure_dict["median_map"].show()

In [ ]:
plotter = elq_model.plot_log_posterior(burn_in_value=burn_in, plot=plotter)
plotter.figure_dict["log_posterior_plot"].show()

In [ ]:
plotter = elq_model.components["source"].plot_iterations(plot=plotter, burn_in_value=burn_in, y_axis_type="linear")
plotter = elq_model.components["source"].plot_iterations(plot=plotter, burn_in_value=burn_in, y_axis_type="log")

plotter.figure_dict["estimated_values_plot"].show()
plotter.figure_dict["log_estimated_values_plot"].show()
plotter.figure_dict["number_of_sources_plot"].show()

In [ ]:
plotter = elq_model.components["offset"].plot_iterations(
    plot=plotter, sensor_object=elq_model.sensor_object, burn_in_value=burn_in
)
plotter = elq_model.components["offset"].plot_distributions(
    plot=plotter, sensor_object=elq_model.sensor_object, burn_in_value=burn_in
)
plotter.figure_dict["offset_iterations"].show()
plotter.figure_dict["offset_distributions"].show()

In [ ]:
plotter = elq_model.components["error_model"].plot_iterations(
    plot=plotter, sensor_object=elq_model.sensor_object, burn_in_value=burn_in
)
plotter = elq_model.components["error_model"].plot_distributions(
    plot=plotter, sensor_object=elq_model.sensor_object, burn_in_value=burn_in
)
plotter.figure_dict["error_model_iterations"].show()
plotter.figure_dict["error_model_distributions"].show()

Finally we show all keys of figures present. As said before we could have just created all figures and perform one call to `plotter.show_all()` which shows all figures at once.

In [ ]:
list(plotter.figure_dict.keys())